## Importing Packages

In [104]:
import nibabel as nib
import os,sys
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from tqdm import tqdm
import pandas as pd

## Functions

In [113]:
fix_dir =  lambda x: os.path.abspath(x) + '/'

Nuclei = {
            1: '1-THALAMUS',
            2: '2-AV',
            4: '4-VA',
            5: '5-VLa',
            6: '6-VLP',
            7: '7-VPL',
            8: '8-Pul',
            9: '9-LGN',
            10: '10-MGN',
            11: '11-CM',
            12: '12-MD-Pf',
            13: '13-Hb',
            14: '14-MTT'}


def normalize_decorator(function):
    def wrapper(dir):
        im, imD = function(dir)
        return (im, imD > imD.max() / 2)
    return wrapper


@normalize_decorator
def load_image(dir):
    true = nib.load(dir)
    return ( true , true.get_fdata() )
    

class MeasureMetrics():
    def __init__(self, true, pred):
        # assert isinstance(true, bool)
        # assert isinstance(pred, bool)

        true = true > true.max()/2
        pred = pred > pred.max()/2

        vsi = lambda X, Y: 1 - (np.abs(X.sum() - Y.sum()) / (X.sum() + Y.sum()))
        dice = lambda X, Y: (X * Y).sum() * 2 / (X.sum() + Y.sum() + np.finfo(float).eps)

        def hd(X, Y):
            a = np.zeros((Y.shape[2], 2))
            for i in range(Y.shape[2]):
                a[i, :] = [Y[..., i].sum(), directed_hausdorff(Y[..., i], X[..., i])[0]]

            return np.sum(a[:, 0] * a[:, 1]) / np.sum(a[:, 0])

        self.VSI = vsi(pred, true)
        self.DICE = dice(pred, true)
        self.HD = hd(pred, true)
        self.VOLUME_TRUE = true.sum()
        self.VOLUME_PRED = pred.sum()


def loop_all_nuclei(dir_true: str, dir_pred: str):
    assert isinstance(dir_true, str)
    assert isinstance(dir_pred, str)

    DICE, VSI, HD, VOLUME_TRUE, VOLUME_PRED = [], [], [], [], []
    for nucleus in Nuclei.items():

        true, trueD = load_image(os.path.abspath(dir_true) + '/Label/' + nucleus[1] + '_PProcessed.nii.gz')
        pred, predD = load_image(os.path.abspath(dir_pred) + '/' + nucleus[1] + '.nii.gz')


        resolution = {'true': true.header['pixdim'][1:4], 'pred': pred.header['pixdim'][1:4]}

        metrics_nucleus = MeasureMetrics(trueD, predD)

        DICE.append(metrics_nucleus.DICE)
        VSI.append(metrics_nucleus.VSI)
        HD.append(metrics_nucleus.HD)
        VOLUME_TRUE.append(metrics_nucleus.VOLUME_TRUE * resolution['true'].prod())
        VOLUME_PRED.append(metrics_nucleus.VOLUME_PRED * resolution['pred'].prod())

    return DICE, VSI, HD, VOLUME_TRUE, VOLUME_PRED

def loop_all_subjects(dir_true, dir_pred):

    assert os.path.exists(dir_true) 
    assert os.path.exists(dir_pred)

    SUBJECTS_TRUE = set([v for v in os.listdir(dir_true) if 'vimp' in v])
    SUBJECTS_PRED = set([v for v in os.listdir(dir_pred) if 'vimp' in v])

    DICE, VSI, HD, VOLUME_TRUE, VOLUME_PRED = {}, {}, {}, {}, {}
    for subject in SUBJECTS_PRED.intersection(SUBJECTS_TRUE):

        dir_true_subject = os.path.abspath(dir_true) + '/'  + subject
        dir_pred_subject = os.path.abspath(dir_pred) + '/'  + subject

        DICE[subject], VSI[subject], HD[subject], VOLUME_TRUE[subject], VOLUME_PRED[subject] = loop_all_nuclei(dir_true_subject , dir_pred_subject)


    return DICE, VSI, HD, VOLUME_TRUE, VOLUME_PRED


class UpdatingMetrics():
    """ This class loops over all subjects inside the directory, measure Dice, HD, VSI, Volume, Volume-Manual 
        and merge them into their corresponding dictionary """

    def __init__(self):
        self.DICE_FULL, self.VSI_FULL, self.HD_FULL, self.VOLUME_TRUE_FULL, self.VOLUME_PRED_FULL = {},{},{},{},{}

    def measure(self, dir_true:str, dir_pred:str ):

        DICE, VSI, HD, VOLUME_TRUE, VOLUME_PRED = loop_all_subjects(dir_true, dir_pred)

        self.DICE_FULL.update(DICE)
        self.VSI_FULL.update(VSI)
        self.HD_FULL.update(HD)
        self.VOLUME_TRUE_FULL.update(VOLUME_TRUE)
        self.VOLUME_PRED_FULL.update(VOLUME_PRED)

    def write_xlsx(self, dir_output):
        """ saving the measured metrics into an Exel file """
        
        NAMES = [item[1] for item in Nuclei.items()]
        with pd.ExcelWriter(dir_output) as writer:  
            for values, sheet_name in [ (self.DICE_FULL,'Dice') , (self.VSI_FULL,'VSI') , (self.HD_FULL,'HD') , (self.VOLUME_PRED_FULL,'Volume') , (self.VOLUME_TRUE_FULL,'Volume-Mn') ]:
                pd.DataFrame(values, index=NAMES).T.to_excel(writer, sheet_name=sheet_name)

## WMn-MPRAGE:  Running on All Subjects

In [114]:
UM = UpdatingMetrics()

dir_pred = '/Users/artinmac/GoogleDrive/RESEARCH/projects/Thalamus/Results/2nd-Submission/final-Method-Dice_VSI_HD/Cases/WMn'
for disease_type in ['Main', 'ET']:
    for x in tqdm('a b c d e f g h'.split()):
        
        dir_true = f'/Volumes/Elements/Research/Thalamus/Datasets/{disease_type}/{x}'
        UM.measure(dir_true=dir_true, dir_pred=dir_pred)


# Wrting the results
UM.write_xlsx(dir_output=dir_pred + ' All Metrics.xlsx')

100%|██████████| 8/8 [00:00<00:00, 10958.34it/s]


## CSFn-MPRAGE:  Running on All Subjects

In [115]:
UM = UpdatingMetrics()

dir_pred = '/Users/artinmac/GoogleDrive/RESEARCH/projects/Thalamus/Results/2nd-Submission/final-Method-Dice_VSI_HD/Cases/CSFn'
for x in tqdm('a b c d'.split()):
    dir_true = f'/Volumes/Elements/Research/Thalamus/Datasets/CSFn2/{x}'
    UM.measure(dir_true=dir_true, dir_pred=dir_pred)


# Wrting the results
UM.write_xlsx(dir_output=dir_pred + ' All Metrics.xlsx')

100%|██████████| 4/4 [01:26<00:00, 21.50s/it]


In [ ]:
r